In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy

%matplotlib inline


In [ ]:
from scipy import random
from scipy import stats

In [ ]:
import thermoengine
from thermoengine import chem

        

In [ ]:
chem.oxide_props['oxides']

In [ ]:
# Load BSE composition from Lyubetskaya2007
oxide_names = np.array(['SiO2', 'Al2O3', 'FeO', 'MgO', 'CaO', 'Na2O'] )
elem_names = np.array(['Si', 'Al', 'Fe', 'Mg', 'Ca', 'Na', 'O'] )
MOL_OXIDE_TO_ELEM = np.array([
    [1,0,0,0,0,0,2],
    [0,2,0,0,0,0,3],
    [0,0,1,0,0,0,1],
    [0,0,0,1,0,0,1],
    [0,0,0,0,1,0,1],
    [0,0,0,0,0,2,1],
])
oxide_wts = np.array([60.0848, 101.96128, 71.8464, 40.3044, 56.0794, 61.97894])


In [ ]:
oxide_wts_BSE = np.array([44.95, 3.52, 7.97, 39.50, 2.79, 0.298])

mol_oxides_BSE = oxide_wts_BSE/oxide_wts
mol_oxides_BSE = mol_oxides_BSE /np.sum(mol_oxides_BSE )
mol_oxides_BSE

In [ ]:
mol_elems_BSE = np.dot(mol_oxides_BSE, MOL_OXIDE_TO_ELEM)
mol_elems_BSE = mol_elems_BSE/np.sum(mol_elems_BSE)
mol_elems_BSE

In [ ]:
mol_oxides_Brg = np.array([1,0,0,1,0,0])
mol_oxides_Brg = mol_oxides_Brg/np.sum(mol_oxides_Brg)
mol_elems_Brg = np.dot(mol_oxides_Brg, MOL_OXIDE_TO_ELEM)
mol_elems_Brg = mol_elems_Brg/np.sum(mol_elems_Brg)

mol_elems_Brg

In [ ]:
elem_ratio = mol_elems_BSE/mol_elems_Brg
ind_limit = np.argmin(elem_ratio)
frac_Brg = elem_ratio[ind_limit]
frac_Brg


In [ ]:
mol_elems_Resid = mol_elems_BSE-frac_Brg*mol_elems_Brg
mol_elems_Resid = mol_elems_Resid/(1-frac_Brg)
np.sum(mol_elems_Resid)

In [ ]:
mol_elems_Resid

In [ ]:
elem_names

In [ ]:
mol_cats_BSE = mol_elems_BSE[:-1]/(1-mol_elems_BSE[-1])
mol_cats_Brg = mol_elems_Brg[:-1]/(1-mol_elems_Brg[-1])
mol_cats_Resid = mol_elems_Resid[:-1]/(1-mol_elems_Resid[-1])


print('mol_cats_BSE = ', mol_cats_BSE)
display()
print('mol_cats_Brg = ')
display(mol_cats_Brg)
print('mol_cats_Resid = ')
display(mol_cats_Resid)

In [ ]:
import pandas as pd
mantle_comp_df = pd.DataFrame([
    pd.Series(data=mol_cats_BSE, index=elem_names[:-1]),
    pd.Series(data=mol_cats_Brg, index=elem_names[:-1]),
    pd.Series(data=mol_cats_Resid, index=elem_names[:-1])],
    index=['BSE','Brg','Resid']).T
display(mantle_comp_df)
print('frac_Brg = ', frac_Brg)

In [ ]:
oxide_data = mantlemelt.get_oxide_data()
morb_comp_d = mantlemelt.get_comp(kind='morb')
mantle_comp_d = mantlemelt.get_comp(kind='mantle')
mgsio3_comp_d =  mantlemelt.get_comp(kind='mgsio3')
mgfesio3_comp_d =  mantlemelt.get_comp(kind='mgfesio3')
chondrite_comp_d = mantlemelt.get_comp(kind='chondrite')


In [ ]:

ncat = np.sum(morb_comp_d['catoxy_ratio'])
print('oxy fraction  = ', 1/(1+ncat))
morb_comp_d

In [ ]:
mantle_comp_d

In [ ]:
mantle_comp_d

In [ ]:
mgsio3_comp_d

In [ ]:
compB_d = mgsio3_comp_d
compM_d = mantle_comp_d
compR_d, molR = mantlemelt.get_comp_residual(compM_d, compB_d)
print('moles of residual = ', molR)
compR_d

In [ ]:
X=1
CNwid = 0.5
Smix0, compX_d = mantlemelt.calc_binary_mixing_entropy(0, compR_d, compB_d, CNwid=CNwid)
Smix1, compX_d = mantlemelt.calc_binary_mixing_entropy(1, compR_d, compB_d, CNwid=CNwid)
print(Smix0-Smix1)
compX_d

In [ ]:
(compB_d['mol']+compR_d['mol']*molR)/(1+molR)

In [ ]:
compM_d['mol']

In [ ]:
compB_d['catoxy_ratio']

In [ ]:
compM_d['catoxy_ratio']

In [ ]:
ind_limit = np.argmin(compM_d['mol']-compB_d['mol'])

if compM_d['mol'][ind_limit]< compB_d['mol'][ind_limit]:
    source_fac = compB_d['mol'][ind_limit]/compM_d['mol'][ind_limit]
    compR_mol = source_fac*compM_d['mol']-compB_d['mol']
        
print(source_fac)
print(source_fac-1)
compR_mol *= 100/np.sum(compR_mol)
compR_mol


In [ ]:
compR_mol_d = dict([(ox,molox) for ox, molox in zip(compM_d['oxides'], compR_mol)])
compR_d = mantlemelt.calc_comp_details(compR_mol_d, kind='mol')
compR_d

In [ ]:
CNwid_avg = 1e-6
CNwid_avg = 2
CNwid_avg = 3./4


CNwid_a = CNwid_avg
# CNwid_a = np.array([3./4, .20, .20, .20, .20, .20])

In [ ]:
mixing_binary_mant_d = mantlemelt.calc_mixing_entropy_binary(mgsio3_comp_d,mantle_comp_d,CNwid_a=CNwid_a)
mixing_binary_chon_d = mantlemelt.calc_mixing_entropy_binary(mgsio3_comp_d,chondrite_comp_d,CNwid_a=CNwid_a)
mixing_binary_morb_d = mantlemelt.calc_mixing_entropy_binary(mgsio3_comp_d,morb_comp_d,CNwid_a=CNwid_a)



In [ ]:
mgsio3_comp_d

In [ ]:
mantle_comp_d

In [ ]:
mixing_binary_mant_d['Natom_a']

In [ ]:
mixing_binary_mant_d.keys()

In [ ]:
plt.plot(mixing_binary_mant_d['X_a'],mixing_binary_mant_d['S_a'],'k-')
dS_mant = mixing_binary_mant_d['S_a'][-1]-mixing_binary_mant_d['S_a'][0]
plt.ylabel('Entropy of Mixing')
plt.xlabel('Composition')
ax = plt.gca()
xticks = np.array([0,.2,.4,.6,.8,1.0])
xticklbls = xticks.tolist()
xticklbls[0] = 'MgSiO3'
xticklbls[-1] = 'Pyrolite'
ax.set_xticks(xticks)
ax.set_xticklabels(xticklbls)
plt.title('dS = '+str(dS_mant))


In [ ]:
Vratio = 0.7
idealgas_binary_chon_d = mantlemelt.calc_idealgas_entropy_binary(mgsio3_comp_d,chondrite_comp_d,Vratio=Vratio)

plt.plot(idealgas_binary_chon_d['X_a'],idealgas_binary_chon_d['S_a'],'k-',
         idealgas_binary_chon_d['X_a'],idealgas_binary_chon_d['S_partialA_a'],'r--')

print(idealgas_binary_chon_d['dS_partialA'])
print(idealgas_binary_chon_d['dS_partialA']/1.5)

In [ ]:
mixing_binary_mgfepv_chon_d = mantlemelt.calc_mixing_entropy_binary(mgfesio3_comp_d,chondrite_comp_d,CNwid_a=CNwid_a)
plt.plot(mixing_binary_mgfepv_chon_d['X_a'],mixing_binary_mgfepv_chon_d['S_a'],'k-',
         mixing_binary_mgfepv_chon_d['X_a'],mixing_binary_mgfepv_chon_d['S_partialA_a'],'r--')

print(mixing_binary_mgfepv_chon_d['dS_partialA'])
print(mixing_binary_mgfepv_chon_d['dS_partialA']/1.5)


In [ ]:
plt.plot(mixing_binary_chon_d['X_a'],mixing_binary_chon_d['S_a'],'k-',
         mixing_binary_chon_d['X_a'],mixing_binary_chon_d['S_partialA_a'],'r--')

print(mixing_binary_chon_d['dS_partialA'])
print(mixing_binary_chon_d['dS_partialA']/1.5)

In [ ]:
plt.plot(mixing_binary_mant_d['X_a'],mixing_binary_mant_d['S_a'],'k-',
         mixing_binary_mant_d['X_a'],mixing_binary_mant_d['S_partialA_a'],'r--')
print(mixing_binary_mant_d['dS_partialA'])
print(mixing_binary_mant_d['dS_partialA']/1.5)

In [ ]:
plt.plot(mixing_binary_morb_d['X_a'],mixing_binary_morb_d['S_a'],'k-',
         mixing_binary_morb_d['X_a'],mixing_binary_morb_d['S_partialA_a'],'r--')

print(mixing_binary_morb_d['dS_partialA'])
print(mixing_binary_morb_d['dS_partialA']/1.5)

In [ ]:
# Calculate tangent for partial molar quantity
X_a = mixing_binary_mant_d['X_a']
X_mantle = 1-.01*frac_mgsio3_cat_mantle
dX = X_a[1]-X_a[0]
dSdX_a = np.gradient(S_a,dX)
dSdX_mant = np.interp(X_mantle,X_a,dSdX_a)
S_mant = np.interp(X_mantle,X_a,S_a)
S_partial_mgsio3_a = S_mant+dSdX_mant*(X_a-X_mantle)

dS_partial = S_partial_mgsio3_a[0]-S_a[0]
print(dS_partial)


plt.plot(X_a,S_a,'k-')
plt.plot(X_a,S_partial_mgsio3_a,'r--')
ylim_a = plt.ylim()
plt.plot([X_mantle,X_mantle],ylim_a,'-',color=[.7,.7,.7])
plt.xlim(-.05,1.05)
plt.xlim(0,1)
xticks_a = plt.xticks()[0]
xticklbl = [str(x) for x in xticks_a]
xticklbl[0] = '0.0\n' r'MgSiO$_3$'
xticklbl[-1] = '1.0\n' r'Pyr-MgSiO$_3$'
xticks_all_a = np.hstack((xticks_a,X_mantle))
xticklbl.append('\nmantle')
xticklbl_a = np.array(xticklbl)
inds_a = np.argsort(xticks_all_a)
# print(xticks_all_a[inds_a])
# print(xticklbl_a[inds_a])
# plt.xticks(np.hstack((xticks_a,X_mantle)),xticklbl.append('mantle'))
plt.xticks(xticks_all_a[inds_a],xticklbl_a[inds_a])
plt.ylim(ylim_a)
# plt.ylim(0.5,1.75)
plt.xlabel('Composition',fontsize=15)
plt.ylabel(r'$S_{\rm {mix}} / N k_B$',fontsize=20)
plt.savefig('figs/mantle-mixing-entropy-ideal-cation.png',bbox_inches='tight',dpi=450)

In [ ]:
# Calculate tangent for partial molar quantity
dX = X_a[1]-X_a[0]
dSdX_a = np.gradient(S_a,dX)
dSdX_mant = np.interp(1.0,X_a,dSdX_a)
dSdX_mant = dSdX_a[-1]
S_mant = np.interp(1.0,X_a,S_a)
S_mant = S_a[-1]
S_partial_mgsio3_a = S_mant+dSdX_mant*(X_a-1.0)

dS_partial = S_partial_mgsio3_a[0]-S_a[0]
print(dS_partial)


plt.plot(X_a,S_a,'k-')
plt.plot(X_a,S_partial_mgsio3_a,'r--')
ylim_a = plt.ylim()
plt.xlim(-.05,1.05)
plt.xlim(0,1)
xticks_a = plt.xticks()[0]
xticklbl = [str(x) for x in xticks_a]
xticklbl[0] = '0.0\n' r'MgSiO$_3$'
xticklbl[-1] = '1.0\n' r'Mantle'
# print(xticks_all_a[inds_a])
# print(xticklbl_a[inds_a])
# plt.xticks(np.hstack((xticks_a,X_mantle)),xticklbl.append('mantle'))
plt.xticks(xticks_a,xticklbl)
plt.ylim(ylim_a)
# plt.ylim(0.5,1.75)
plt.xlabel('Composition',fontsize=15)
plt.ylabel(r'$S_{\rm {mix}} / N k_B$',fontsize=20)
plt.savefig('figs/mantle-mixing-entropy-ideal-cation.png',bbox_inches='tight',dpi=450)

In [ ]:
# Vratio_max = .7 # leads to ideal gas component similar to entropy of mixing component
Vratio_max = .7
Vratio_max = .6
Vratio_a = (1-X_a)*1 + X_a*Vratio_max
dS_gas_a = np.log(Vratio_a)
dSdX_gas_a = np.gradient(dS_gas_a,dX)

dSdX_gas_mant = dSdX_gas_a[-1]
S_gas_mant = dS_gas_a[-1]
S_partial_gas_mgsio3_a = S_gas_mant+dSdX_gas_mant*(X_a-1.0)
dS_partial_gas = S_partial_gas_mgsio3_a[0]-dS_gas_a[0]


print(dS_partial_gas)

plt.plot(X_a,dS_gas_a,'k-')
plt.plot(X_a,S_partial_gas_mgsio3_a,'r--')


In [ ]:
plt.plot(X_a, 1-(1-X_a)/(1-.5*X_a),'r-',X_a,X_a,'k--')

In [ ]:
#
1500/1.3

In [ ]:
2500/1.252500/1.25